## Setup

1. Complete the previous challenges.

In [1]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICE_KEY"

# Function URL
function_url = 'YOUR_FUNCTION_URL'

## Function to wrap HTTP requests

In [2]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


## Synonym Map

In [4]:
import json

body = {
   "format":"solr",
   "synonyms": "USA, United States, America, United States of America\n" \
      "UK, GB, United Kingdom, Great Britain, Britain\n" \
      "UAE, United Arab Emirates, Emirates\n"
}

try:
    result = azsearch_rest(request_type="PUT", function_name="synonymmaps/locationsynonyms", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Update the Index

In [5]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The sentiment score
      "name": "sentiment",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": True
    },
    {
      # Key phrases that might help identify the subject of the text
      "name": "key_phrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # Geographical locations mentioned in the text
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # People mentioned in the text
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "links",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },    {
      # The image content in the document
      "name": "image_descriptions",
      "type": "Collection(Edm.String)",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The text contained in images in the document
      "name": "image_text",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The combined source text plus the text extracted from images
      "name": "merged_text",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": False,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The most common words in the text
      "name": "top_words",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    }
  ],
    "suggesters": [
    {
      "name": "location_suggester",
      "searchMode": "analyzingInfixMatching",
      "sourceFields": ["locations"]
    }
  ],
  "scoringProfiles": [  
    {  
      "name": "boost_relevance",  
      "text": {  
        "weights": {  
          "key_phrases": 5.0 
        }
      },
          "functions": [  
            {  
              "type": "freshness",  
              "fieldName": "last_modified",  
              "boost": 2,  
              "interpolation": "quadratic",  
              "freshness": {  
                "boostingDuration": "P365D"  
              }  
            },  
            {
              "type": "magnitude",  
              "fieldName": "size",  
              "boost": 2,  
              "interpolation": "linear",  
              "magnitude": {  
                "boostingRangeStart": 1000,  
                "boostingRangeEnd": 500000,  
                "constantBoostBeyondRange": "true"
              }  
            }
          ]
      }
  ]
}

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Update the Skillset

In [7]:
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
       # Detect the language
      {
          "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
          "inputs": [
            {
              "name": "text", "source": "/document/content"
            }
          ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "languageCode"
                }
            ]
        },
      # Generate text description from images (including those embedded in documents)
        {
            "description": "Extract descriptions from images.",
            "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "visualFeatures": ["Description"],
            "details": ["Landmarks"],
            "inputs": [
              {
                "name": "image",
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "description",
                "targetName": "imageDescription"
              }
            ]
        },
      # Extract text from images (including those embedded in documents)
        {
            "description": "Extract text from images.",
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
              {
                "name": "image", 
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "text"
              }
            ]
        },
       # Merge the extracted image text back into the content at the appropriate locations
        {
          "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
          "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
          "context": "/document",
          "insertPreTag": "[",
          "insertPostTag": "]",
          "inputs": [
            {
              "name":"text", "source": "/document/content"
            },
            {
              "name": "itemsToInsert", "source": "/document/normalized_images/*/text"
            },
            {
              "name":"offsets", "source": "/document/normalized_images/*/contentOffset" 
            }
          ],
          "outputs": [
            {
              "name": "mergedText"
            }
          ]
        },
         # Detect the sentiment
        {
          "description": "Detect sentiment.",
          "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
        "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
            "outputs": [
                {
                  "name": "score",
                  "targetName": "sentimentScore"
                }
            ]
        },
       # Extract entities
        {
          "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
          "categories": [ "Organization", "location", "person", "datetime", "url" ],
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "locations"
            },
            {
              "name": "persons"
            },
            {
              "name": "urls"
            },
            {
              "name": "entities"
            }
          ]
        },
      # Extract key phrases from the pages
      {
          "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
          "context": "/document",
          "defaultLanguageCode": "en",
          "maxKeyPhraseCount": 5,
          "inputs": [
            {
              "name": "text", "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "keyPhrases"
            }
          ]
        }, 
       # Use a custom skill to find the most common words in each document
          {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "custom skill",
            "uri": function_url,
            "batchSize":1,
            "context": "/document",
            "inputs": [
              {
                "name":"language",
                "source": "/document/languageCode"
              },
              {
                "name": "text",
                "source": "/document/mergedText"
              }
            ],
            "outputs": [
              {
                "name": "text",
                "targetName": "topWords"
              }
            ]
          },
        # Shaper skill to create knowledge store projections
          {
          "@odata.type": "#Microsoft.Skills.Util.ShaperSkill",
          "context": "/document",
          "description": "Prepare projection fields",
          "inputs": [
            {
              "name": "file_id",
              "source": "/document/id"
            },
            {
              "name": "file_name",
              "source": "/document/file_name"
            },
            {
              "name": "url",
              "source": "/document/url"
            },
            {
              "name": "top_words",
              "source": "/document/topWords/*"
            },
            {
              "name": "sentiment_score",
              "source": "/document/sentimentScore"
            },
            {
              "name": "locations",
              "source": "/document/locations/*"
            }
          ],
          "name": "projection_shaper",
          "outputs": [
            {
              "name": "output",
              "targetName": "knowledge_projection"
            }
          ]
        }
      ],
      "cognitiveServices":
        {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
        },
        "knowledgeStore":
        { 
        "storageConnectionString": blob_connection_string, 
        "projections": [ 
            { 
                # Create tables
                "tables": [  
                 {
                    # Table with base document fields
                   "tableName": "WebDocs",
                   "generatedKeyName": "document_id", # Generate a key field
                   "source": "/document/knowledge_projection" # Include all base fields in projection
                 }, 
                 {
                     # Table with top_words
                    "tableName": "TopWords",
                    "generatedKeyName": "topword_id",
                    "sourceContext": "/document/knowledge_projection/top_words/*", # Create a record for each top_word
                    "inputs":[
                        # These are the fields in the table
                        {"name": "word", "source": "/document/knowledge_projection/top_words/*"}] 
                 }, 
                 {
                     # Table with sentiment scores
                    "tableName": "SentimentScores",
                    "generatedKeyName": "sentiment_id",
                    "sourceContext": "/document/knowledge_projection/sentiment_score", # Create a record for each sentiment
                    "inputs":[
                       {"name": "sentiment", "source": "/document/knowledge_projection/sentiment_score"}]
                 }, 
                 {
                     # Table with locations
                    "tableName": "Locations",
                    "generatedKeyName": "location_id",
                    "sourceContext": "/document/knowledge_projection/locations/*", # Create a record for each location
                    "inputs":[
                        # These are the fields in the table
                        {"name": "location", "source": "/document/knowledge_projection/locations/*"}] 
                 }
                ], 
                "objects": [ 

                ]      
            },
            {
                # Now create a separate projection for JSON blobs
                "tables": [],
                "objects": [
                    {
                        "storageContainer": "knowledge-data",
                        "source": "/document/knowledge_projection"
                    }
                ]
            }
        ]     
    } 
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/website-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


## Update the Indexer

In [8]:
import json

body = {
  "name":"website-docs-indexer", 
  "dataSourceName" : "website-docs",
  "skillsetName" : "website-skillset",
  "targetIndexName" : "website-docs-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from cognitive skills to index fields
        {
            "sourceFieldName": "/document/normalized_images/*/imageDescription", 
            "targetFieldName": "image_descriptions"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/text",
            "targetFieldName": "image_text"
        },
        {
            "sourceFieldName": "/document/mergedText", 
            "targetFieldName": "merged_text"
        },
        {
            "sourceFieldName": "/document/sentimentScore", 
            "targetFieldName": "sentiment"
        },
        {
          "sourceFieldName" : "/document/persons", 
          "targetFieldName" : "people"
        },
        {
          "sourceFieldName" : "/document/locations", 
          "targetFieldName" : "locations"
        },
        {
          "sourceFieldName" : "/document/urls", 
          "targetFieldName" : "links"
        },
        {
          "sourceFieldName" : "/document/entities", 
          "targetFieldName" : "entities"
        },
        {
          "sourceFieldName" : "/document/keyPhrases/*", 
          "targetFieldName" : "key_phrases"
        },
        {
          "sourceFieldName" : "/document/topWords", 
          "targetFieldName" : "top_words"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata",
        "imageAction": "generateNormalizedImages"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/website-docs-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [9]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [10]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Create Wrapper Function for Query Requests

In [11]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### View Top Words

In [15]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text, top_words',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
   # print(result)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        for word in value['top_words']:
            print(word)

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Hits: 15

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/collateral/Las%20Vegas%20Brochure.pdf
Las Vegas Brochure.pdf
vegas
las
city
hotel
margie’s
travel
known
populated
nevada
county

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201848.pdf
201848.pdf
hotel
las
vegas
check
room
live
reputation
fountain
usa
first

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201857.pdf
201857.pdf
amazing
experience
canal
hotel
las
vegas
usa
expected
something
top

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201845.pdf
201845.pdf
hotel
great
room
beds
volcano
las
vegas
view
also
comfy

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201847.pdf
201847.pdf
place
free
good
las
vegas
luxurious
world
u
fresh
every

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201850.pdf
201850.pdf
experience
hotel
las
vegas


### Filter Based on Top Words

In [16]:
import urllib.parse, json

search_terms = input("Search: ")
filter_word = input("Keyword: ").lower()

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'{0}'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text, top_words',
    '$filter':"top_words/any(w: w eq '{0}')".format(filter_word),
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        for word in value['top_words']:
            print(word)

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Keyword: nice
Hits: 2

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201841.pdf
201841.pdf
hotel
rooms
location
volcano
also
awesome
nice
excellent
las
vegas

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201844.pdf
201844.pdf
hotel
nice
rooms
top
enough
volcano
las
vegas
usa
really
